In [5]:
import csv

In [1]:
to_save_dir = '../../dgl/apps/kg/data/wiki/'

In [2]:
raw_dir = '../data/raw/'

In [28]:
with open('../../dgl/apps/kg/data/FB15k/train.txt', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)
        break

['/m/027rn\t/location/country/form_of_government\t/m/06cx9']


In [7]:
# file_inp = open(input_file,'r')
# file_out = open(output_file,'w') 

# inp = csv.reader(file_inp)
# #out = csv.writer(file_out, delimiter=' ')

In [82]:
entity_input_file = raw_dir + 'wikidata_20190805.item.csv'
entity_output_file = to_save_dir + 'entities.dict'
entity_keep_cols = [0, 3]
entity_text_format = "{}\t{}\n"
entity_skip_lines = 1 
entity_dict = {}

In [83]:
edge_input_file = raw_dir + 'wikidata_20190805.property.csv'
edge_output_file = to_save_dir + 'relations.dict'
edge_keep_cols = [0, 1]
edge_text_format = "{}\t{}\n"
edge_skip_lines = 1 
edge_dict = {}

In [84]:
def write_to_file_with_format(input_file, output_file, keep_cols, text_format, skip_lines = 0):
    m = len(keep_cols)
    with open(input_file, 'r') as file_inp, open(output_file, 'w') as file_out:
        inp = csv.reader(file_inp)
        for _ in range(skip_lines):
            next(inp)
        for l in inp:
            to_write = [l[col] for col in keep_cols if l[col]]
            if len(to_write) < m:
                continue
            file_out.write(text_format.format(*to_write))

In [85]:
def write_to_file_with_format_and_dict(input_file, output_file, keep_cols, text_format, skip_lines = 0, dictionary = None):
    m = len(keep_cols)
    
    i = 0
    mapping_dict = {}
    with open(input_file, 'r') as file_inp, open(output_file, 'w') as file_out:
        inp = csv.reader(file_inp)
        for _ in range(skip_lines):
            next(inp)
        for l in inp:
            to_write = [l[col] for col in keep_cols if l[col]]
            original_key = int(to_write[0])
            to_write[0] = i
            if len(to_write) < m:
                continue
            mapping_dict[original_key] = i
            i += 1
            file_out.write(text_format.format(*to_write))
            dictionary[original_key] = to_write[1]
    return mapping_dict

In [86]:
# write_to_file_with_format(
#     input_file=edge_input_file,
#     output_file=edge_output_file,
#     keep_cols=edge_keep_cols,
#     text_format=edge_text_format,
#     skip_lines=edge_skip_lines,
# )

edge_mapping_dict = write_to_file_with_format_and_dict(
    input_file=edge_input_file,
    output_file=edge_output_file,
    keep_cols=edge_keep_cols,
    text_format=edge_text_format,
    skip_lines=edge_skip_lines,
    dictionary=edge_dict,
)

In [88]:
# write_to_file_with_format(
#     input_file=entity_input_file,
#     output_file=entity_output_file,
#     keep_cols=entity_keep_cols,
#     text_format=entity_text_format,
#     skip_lines=entity_skip_lines,
# )

entity_mapping_dict = write_to_file_with_format_and_dict(
    input_file=entity_input_file,
    output_file=entity_output_file,
    keep_cols=entity_keep_cols,
    text_format=entity_text_format,
    skip_lines=entity_skip_lines,
    dictionary=entity_dict,
)

In [95]:
len(entity_dict)

8059838

In [94]:
max(entity_mapping_dict.values())

8059837

In [96]:
len(edge_dict)

6490

In [97]:
max(edge_mapping_dict.values())

6489

In [98]:
14591*14951

218150041

In [61]:
import numpy as np

In [108]:
!head ../data/wikipages_triplets.csv

source_item_id,edge_property_id,target_item_id,el_rank
1,793,323,1
1,793,837317,1
1,793,1208634,1
1,793,1079826,1
1,793,273508,1
1,793,1079806,1
1,793,3491753,1
1,31,36906466,1
1,910,5551050,1


In [112]:
def write_to_file_with_format_for_triplet(input_file, output_file, output_file2, output_file3, text_format,\
                                          skip_lines = 0, triplet_head_idx = 0, triplet_tail_idx = 2, \
                                         triplet_edge_idx = 1):
    with open(input_file, 'r') as file_inp, open(output_file, 'w') as file_out,\
    open(output_file2, 'w') as file_out2, open(output_file3, 'w') as file_out3:
        inp = csv.reader(file_inp)
        for _ in range(skip_lines):
            next(inp)
        for l in inp:
            to_write = []
            triplet_head, triplet_edge, triplet_tail = \
            int(l[triplet_head_idx]), int(l[triplet_edge_idx]), int(l[triplet_tail_idx])
            if not triplet_head in entity_mapping_dict or not triplet_tail in entity_mapping_dict or \
            not triplet_edge in edge_mapping_dict:
                continue
            rand_int = np.random.randint(200)
            if rand_int > 1:
                write_file_out = file_out
            elif rand_int == 0:
                write_file_out = file_out2
            else:
                write_file_out = file_out3
            #write_file_out.write(text_format.format(entity_mapping_dict[triplet_head], edge_mapping_dict[triplet_edge], entity_mapping_dict[triplet_tail]))
            write_file_out.write(text_format.format(entity_dict[triplet_head], edge_dict[triplet_edge], entity_dict[triplet_tail]))

In [116]:
!head ../data/wikipages_triplets.csv

source_item_id,edge_property_id,target_item_id,el_rank
1,793,323,1
1,793,837317,1
1,793,1208634,1
1,793,1079826,1
1,793,273508,1
1,793,1079806,1
1,793,3491753,1
1,31,36906466,1
1,910,5551050,1


In [126]:
entity_mapping_dict[1]

4083730

In [118]:
entity_dict[1]

'Universe'

In [119]:
entity_dict[323]

'Big Bang'

In [113]:
triplet_input_file = '../data/wikipages_triplets.csv'#raw_dir + 'wikidata_20190805.qpq_item_statements.csv'
triplet_output_file = to_save_dir + 'train.txt'
triplet_output_file_2 = to_save_dir + 'valid.txt'
triplet_output_file_3 = to_save_dir + 'test.txt'
triplet_text_format = "{}\t{}\t{}\n"
triplet_skip_lines = 1

In [114]:
write_to_file_with_format_for_triplet(
    input_file=triplet_input_file,
    output_file=triplet_output_file,
    output_file2=triplet_output_file_2,
    output_file3=triplet_output_file_3,
    text_format=triplet_text_format,
    skip_lines=triplet_skip_lines,
)